# Advent of Code 2024
## Day 15
*<https://adventofcode.com/2024/day/15>*

In [1]:
import heapq
import math
import re
import functools as ft
from collections import Counter, defaultdict, deque, namedtuple
from itertools import combinations, permutations, product
from string import ascii_letters, ascii_lowercase, ascii_uppercase

# import z3
from rich import inspect, pretty, print

from new_helper import *

pretty.install()

try:
    import ipykernel

    with open("kernel", "w+") as f:
        f.write(ipykernel.get_connection_file())
except:
    pass

In [2]:
DAY = 15
YEAR = 2024
input_str = get_aoc_input(DAY, 2024)
part_1 = part_2 = 0

In [3]:
inp = input_str.parse_groups()

In [4]:
g = Grid(inp[0])
ins = "".join(inp[1])

x, y = g.find("@")

for d in ins:
    dx, dy = DIRECTIONS[d]
    nx, ny = x + dx, y + dy
    while g[nx, ny] == "O":
        nx += dx
        ny += dy

    if g[nx, ny] == "#":
        continue

    g[x, y] = "."
    g[x + dx, y + dy] = "@"
    if (nx, ny) != (x + dx, y + dy):
        g[nx, ny] = "O"

    x, y = x + dx, y + dy# input_str = sample

print(g.pretty(sep_col=""))

|           1111111111222222222233333333334444444444
    | 01234567890123456789012345678901234567890123456789
----+---------------------------------------------------
  0 | ##################################################
  1 | #OOOOOOO..OOOOO..#O..OO#OO#OOO...OOOOOOOO......O##
  2 | #O#OO.......#........O..OOOOOO.....OO........O.OO#
  3 | #....OO#O.....O.....#O.....OOO.#...O.........OOOO#
  4 | #....#OOO.....#............O#O.#....#O.....#O..OO#
  5 | #...O.#OO.........#.O....#OO...O....OO#.....OO#O##
  6 | ##..#..#O.....OO...OO...OOOO........O#..#.#.O#OOO#
  7 | #......#O..@.#.........OO..........OOOO.O.......O#
  8 | #...#..#..........OOO.O......O#.....O#OO...#....O#
  9 | #......OOO........#...##.O..OO......OOO.........O#
 10 | #........OO#.......O.....O..O#...#....OO.........#
 11 | #OO.......#OO.......O.....OOOOOO.....OOO#........#
 12 | #O#....#.....#O......O....#.O..##...OOOO.O....O..#
 13 | #OOO...O......O#...O..OO....O.O......#.O.OO.O....#
 14 | #OOO.......OO..O...O.....O...O.........OOOOOOOO.O#
 15 | #OO#O.....O..........OO........OOOOOO..OOO.OO...O#
 16 | #OOO.#..OO......O.....##OO........O#...#.#......O#
 17 | #OO.....OO.#..........OOO#O..O....OO...O#.#.....O#
 18 | #OO.#....#.........O...OOOO#.O#O..O#O#.OO#O...OOO#
 19 | #OOO.....O...............O.O..##....OO.......O#OO#
 20 | #OO.....O.O.....O.........O..................OOOO#
 21 | #O..#.....#.......O.O.....O..................O...#
 22 | #O....O.........OOO..O...#.........O.........O...#
 23 | #O....#O#O.....#OOO......O.#...#OO....#...#O.....#
 24 | ##....O#O#....O.OOO#....O#.....O##O....O..O......#
 25 | #O.....OO...O##OO...#O..O.........O..........O##O#
 26 | #O...#.....OOO#OO....O.O...#...#..O.....O.....O###
 27 | #O#.O.....O##O.OO.....OO........OOOO.O.....O...OO#
 28 | #OO...#...O.O..OO...O.OO........OOOOOO#...O.O#.O.#
 29 | #.O........OO#OOO..#...OOOO.OOOO.OO..#O........#O#
 30 | #O.O.......O.#.O..O.O...OO..O#O............#O.OOO#
 31 | #O.........OOOO..OOO...O.O....OO#..O.....OO###.#.#
 32 | #OO........O......O......#...O#O#.......OOOO.O...#
 33 | #OO.....O#.O...O..#O......O..#.......O..O.....#.O#
 34 | #O.....#OOO.O..OO.OO#....OOOOO..#..#....OO.OOO...#
 35 | #O...O..OO......OO.OO....OO#OO..#..OO....O.....#O#
 36 | #O........#O...O#.......O.#OO.....#OO....O.OOO#..#
 37 | ##............#.O.........#OO............OO#.....#
 38 | #O................O.......OO#...O#....#O.O.OO.O..#
 39 | #O..........................#O..O......#.O#..O.#.#
 40 | #....O......#..#...OO#......O.OO....#..O#.O.O..O.#
 41 | #..O.......O..O.##OOO.O.#....O...O#...OO...#.O.O.#
 42 | #..O....OO.O.#O.OOOO..O........O..#...O....OO..#.#
 43 | #OOO.#..OO#..O....O........O..#OO.O#.O....O..O...#
 44 | #OO...OO......#...O.O......O#O.O.O..........#....#
 45 | #.................O#...O#...O##..#....O#.....OO..#
 46 | #O............O#.......#OO......#O....#O.........#
 47 | #O..O#O...O#.O...........O##........#.##...#.....#
 48 | #O..O#OO........O....O#OOO.....OO.....#..O.O.OO.O#
 49 | ##################################################

In [5]:
for x, y in g.find_all("O"):
    part_1 += 100 * y + x

In [6]:
g = Grid([l.replace("#", "##").replace("O", "[]").replace(".", "..").replace("@", "@.") for l in inp[0]])

sx, sy = g.find("@")
x, y = sx, sy

for d in ins:
    dx, dy = DIRECTIONS[d]
    Q = deque()
    Q.append((sx + dx, sy + dy))
    to_move = []
    seen = set()

    wall = False
    while Q:
        x, y = Q.pop()
        if g[x, y] == "#":
            wall = True
            break

        if g[x, y] == ".":
            continue

        to_move.append((x, y))

        if (x + dx, y + dy) not in seen:
            Q.append((x + dx, y + dy))
            seen.add((x + dx, y + dy))
        if g[x, y] == "[" and (x + 1, y) not in seen:
            Q.append((x + 1, y))
            seen.add((x + 1, y))
        if g[x, y] == "]" and (x - 1, y) not in seen:
            Q.append((x - 1, y))
            seen.add((x - 1, y))

    if wall:
        continue

    ng = Grid.of_size(g.width, g.height, ".")
    for (x, y), c in g.items():
        if (x, y) not in to_move:
            ng[x, y] = c

    for x, y in to_move:
        ng[x + dx, y + dy] = g[x, y]

    ng[sx, sy] = "."
    ng[sx + dx, sy + dy] = "@"

    g = ng
    sx += dx
    sy += dy

print(g.pretty(sep_col=""))

|           111111111122222222223333333333444444444455555555556666666666777777777788888888889999999999
    | 0123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789
----+-----------------------------------------------------------------------------------------------------
  0 | ####################################################################################################
  1 | ##[]....[]......[][]..[].[].......##[][]..[][]##[][]##[][].[]....[].[]......[]..................####
  2 | ##..##..[]......[]..[][]##.......[].[][][][][][][][]..............[][]....[][]..........[]......[]##
  3 | ##..[][][]....##.[][].........[][][][]..##[][][][]............##....[]......[][]..[][]......[][]..##
  4 | ##..[]....##....[]..........##[][]......[][]..[]........##....##[]..[][]##[]......[][]##........[]##
  5 | ##..........##..............[].[]...##[]..........##............[][][][]..[]##....[]..[]....##..####
  6 | ####....##....##..........[].[]....[]............[]....[]..........[][][].##....##..##....##......##
  7 | ##[][][]..[]..##..........##.......[]........[][][][]...............[][]........[]....[]........[]##
  8 | ##[]..[]##....##..@..............[].........[]............[]##[].....[]...##..........##[][][][]..##
  9 | ##[]....[]..[].....[]...............##......####......[][][][][]......................[]....[]..[]##
 10 | ##.....[][].........[]##.............................[].[]##......##..[][].............[].[]....[]##
 11 | ##............[]....##...[]...........[]...............[]..........[][].........##..[]..[][]......##
 12 | ##..##........##..........##..[].....[].............##........####.[]..[]...........[]..[]..[]....##
 13 | ##....[].....[]..[][]...[]....##[]..............[]..[][][]....[][]........##.......[].............##
 14 | ##....[]................[]...[].[]......[].....[]............[]..[].....[]....[]...[].............##
 15 | ##....##..[][][]........[]..[]..[]......[]..[][].[].........[]...........[].[][]............[]....##
 16 | ##........##..[][]..[]....[][]...[].....[]..####[][]....[].[][].......##[][][]##[]##..............##
 17 | ##..................[]##..[][].........[].[][][][]##...[][][][].............[]..##..##..[]...[][].##
 18 | ##....[]##........##.[][][]......[]....[]...[][][][][]##[]..##..[]..[]##[]##[][]..##[]..[]........##
 19 | ##..............[]...[][].[].....[].......[]..[]....[]......####..[][]..[][][][]....[]......##..[]##
 20 | ##............[]....[][]..[].....[]................[]....[].[]..........[][][][]...[].[]......[][]##
 21 | ##[]..[]##..[]......##..............[]..............[]............................................##
 22 | ##[].......[].......[]..............[]............##[].......[]........[].................[][]..[]##
 23 | ##[]...[]...##..##............##.....[]...............##......##...[].......##......##....[][]....##
 24 | ####[]......[]##[]##.......[].........##..........##[][].....[].####[]..............[]..[][]......##
 25 | ##[][]..................[]####........[]##.....................[]..[]....[]..........[][].[]####[]##
 26 | ##..[]....##..[]........[][]##[].....[].........[]....##......##[][]........................[]######
 27 | ##[]##...[].........[]####.[]...................[]..........[][].......[]..[]........[].[]....[][]##
 28 | ##[][]....[]##......[]..[]...[]..[]......................[][]...........[]..##............##..[]..##
 29 | ##.[][]....[].......[]....##[].[].....##...............[][]...............##..................##[]##
 30 | ##[]............[].[].....##..[][].....[]...............[]##...[][]...........[][]....##....[][][]##
 31 | ##............................[][].......[].......[]......[]....##.[]..[].......[]....######..##..##
 32 | ##............................[][]......[][]......##[]....[]##..##.[].........[][]...[][].[]......##
 33 | ##[].........[]...##....[].....[][].##[][][]..............##........[]..........[][]........##..[][0

In [7]:
for x, y in g.find_all("["):
    part_2 += 100 * y + x

In [8]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 1371036

Part 2: 1392847